In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vqa-dataset/vqa_dataset/data-00016-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00004-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/state.json
/kaggle/input/vqa-dataset/vqa_dataset/data-00011-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/dataset_info.json
/kaggle/input/vqa-dataset/vqa_dataset/data-00002-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00028-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00015-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00013-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00019-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00027-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00031-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00006-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00033-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00008-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00025-of-00034.arrow

In [2]:
from datasets import load_from_disk

In [3]:
dataset = load_from_disk("/kaggle/input/vqa-dataset/vqa_dataset")

Loading dataset from disk:   0%|          | 0/34 [00:00<?, ?it/s]

In [4]:
print(dataset[0])

{'question_type': 'what is this', 'multiple_choice_answer': 'net', 'answers': [{'answer': 'net', 'answer_confidence': 'maybe', 'answer_id': 1}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 2}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 3}, {'answer': 'netting', 'answer_confidence': 'yes', 'answer_id': 4}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 5}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 6}, {'answer': 'mesh', 'answer_confidence': 'maybe', 'answer_id': 7}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 8}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 9}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 10}], 'image_id': 458752, 'answer_type': 'other', 'question_id': 458752000, 'question': 'What is this photo taken looking through?', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x7E262DEB68F0>}


In [5]:
pip install torch torchvision transformers

Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch
import torch.nn as nn
from transformers import ViTImageProcessor,ViTModel, BertModel, BertTokenizer
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Load pretrained models and tokenizers
model_name = 'google/vit-base-patch16-224-in21k'
vit_model = ViTModel.from_pretrained(model_name)  # Consider using ViTForQuestionAnswering if available

# Use ViTImageProcessor instead of the deprecated ViTFeatureExtractor
feature_extractor = ViTImageProcessor.from_pretrained(model_name)
text_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

2024-05-18 12:26:10.187411: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 12:26:10.187513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 12:26:10.318385: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

# Define image transformations
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224x224 pixels
    transforms.ToTensor(),          # Convert image to PyTorch tensor (0-1 range)
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize image
])

# Define the VQADataset class
class VQADataset(Dataset):
    def __init__(self, dataset, vit_model, bert_tokenizer, image_transform, vit_feature_extractor):
        self.dataset = dataset
        self.vit_model = vit_model
        self.bert_tokenizer = bert_tokenizer
        self.image_transform = image_transform
        self.vit_feature_extractor = vit_feature_extractor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image_path = item["image"]
        question = item["question"]
        label = item["multiple_choice_answer"]

        # Preprocess image
        if image_path.mode != "RGB":
            image = image_path.convert("RGB")
        else:
            image = image_path

        # Preprocess image
        image = self.image_transform(image)

        # Tokenize inputs
        inputs = self.bert_tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=64)
        input_ids = inputs["input_ids"].squeeze(0)
        attention_mask = inputs["attention_mask"].squeeze(0)

        # Extract image features using ViT
        with torch.no_grad():
            inputs = self.vit_feature_extractor(images=image, return_tensors="pt",do_rescale=False)
            pixel_values = inputs.pixel_values.to('cuda')  # Move to GPU
            image_features = self.vit_model(pixel_values=pixel_values, return_dict=True).last_hidden_state[:, 0, :].squeeze(0)

        return {
            "image_features": image_features,
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "label": label
        }


In [9]:
class VQAModel(nn.Module):
    def __init__(self, image_model, text_model, vocab_size):
        super(VQAModel, self).__init__()
        self.image_model = image_model
        self.text_model = text_model
        self.hidden_size = 768  # assuming the hidden size of ViT and BERT
        self.image_projection = nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size)
        self.attention_text = nn.Linear(self.hidden_size, 1)
        self.attention_image = nn.Linear(self.hidden_size, 1)
        self.decoder = nn.Linear(in_features=self.hidden_size, out_features=vocab_size)

    def forward(self, image_features, input_ids, attention_mask):
        # Text features
        text_output = self.text_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]

        # Image features projection
        projected_image_features = self.image_projection(image_features)

        # Calculate attention weights
        att_text = torch.sigmoid(self.attention_text(text_output))
        att_image = torch.sigmoid(self.attention_image(projected_image_features))

        # Weighted sum of text and image features
        combined_output = att_text * text_output + att_image * projected_image_features

        # Decoder
        logits = self.decoder(combined_output)
        return logits


In [10]:
# Instantiate model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VQAModel(vit_model, text_model, vocab_size=tokenizer.vocab_size).to(device)

In [11]:
from torch.utils.data import Dataset, DataLoader, random_split
# Create DataLoader instances with custom collate function
from torch.nn.utils.rnn import pad_sequence

small_dataset = dataset.select(range(20000))

# Split the dataset into training, validation, and test sets
train_size = int(0.8 * len(small_dataset))
val_size = int(0.1 * len(small_dataset))
test_size = len(small_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(small_dataset, [train_size, val_size, test_size])

# Create VQADataset instances
train_vqa_dataset = VQADataset(train_dataset, vit_model, tokenizer, image_transform, feature_extractor)
val_vqa_dataset = VQADataset(val_dataset, vit_model, tokenizer, image_transform, feature_extractor)
test_vqa_dataset = VQADataset(test_dataset, vit_model, tokenizer, image_transform, feature_extractor)


def custom_collate_fn(batch):
    # Find maximum sequence length in the batch
    max_seq_length = max(len(item["input_ids"]) for item in batch)
    
    padded_input_ids = []
    padded_attention_masks = []
    images = []
    labels = []
    
    for item in batch:
        # Pad input_ids and attention_mask
        input_ids = torch.tensor(item["input_ids"])
        attention_mask = torch.tensor(item["attention_mask"])
        padded_input_ids.append(input_ids)
        padded_attention_masks.append(attention_mask)
        
        # Add image features
        images.append(item["image_features"])
        
         # Encode labels
        label = item["label"]
        encoded_label = torch.tensor(tokenizer.convert_tokens_to_ids(label))
        labels.append(encoded_label)
        
    
    
    # Pad input_ids and attention_mask
    padded_input_ids = pad_sequence(padded_input_ids, batch_first=True, padding_value=0)
    padded_attention_masks = pad_sequence(padded_attention_masks, batch_first=True, padding_value=0)
    
    # Stack all tensors
    images = torch.stack(images, dim=0)
    
    # Convert labels to tensor
    labels = torch.tensor(labels)  # Assuming labels are numerical
    
    # Check sizes
    assert images.size(0) == len(batch), f"Images tensor size mismatch: Got {images.size(0)}, expected {len(batch)}"
    assert labels.size(0) == len(batch), f"Labels tensor size mismatch: Got {labels.size(0)}, expected {len(batch)}"
    
    return {
        "input_ids": padded_input_ids,
        "attention_mask": padded_attention_masks,
        "image_features": images,
        "label": labels
    }


# Update DataLoader instances with custom collate function
train_loader = DataLoader(train_vqa_dataset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_vqa_dataset, batch_size=32, shuffle=False, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_vqa_dataset, batch_size=32, shuffle=False, collate_fn=custom_collate_fn)


In [12]:
%%time
from tqdm import tqdm
import torch
import torch.nn as nn

# Assuming you have defined train_loader and val_loader

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()
epochs = 15

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        image_features = batch["image_features"].to(device)
        labels = batch["label"]

        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(image_features, input_ids, attention_mask)
        
        # Convert labels to binary tensor
        labels_tensor = torch.zeros_like(outputs)  # Initialize with zeros
        for i, label_indices in enumerate(labels):
            labels_tensor[i, label_indices] = 1  # Set corresponding indices to 1
        
        # Calculate loss
        loss = criterion(outputs, labels_tensor)
        
        # Backward pass
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        total_loss += loss.item()
    
    # Print average loss for this epoch
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}")
    
    # Validation loop
    model.eval()
    val_loss = 0.0
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{epochs}"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            image_features = batch["image_features"].to(device)
            labels = batch["label"]
            
            # Forward pass
            outputs = model(image_features, input_ids, attention_mask)
            
            labels_tensor = torch.zeros_like(outputs)  # Initialize with zeros
            for i, label_indices in enumerate(labels):
                labels_tensor[i, label_indices] = 1  # Set corresponding indices to 1
            
            # Calculate loss
            loss = criterion(outputs, labels_tensor)
            
            val_loss += loss.item()
    
    print(f"Validation Loss: {val_loss/len(val_loader)}")


Epoch 1/15:   0%|          | 0/500 [00:00<?, ?it/s]/tmp/ipykernel_35/3601849629.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(item["input_ids"])
/tmp/ipykernel_35/3601849629.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(item["attention_mask"])
Epoch 1/15: 100%|██████████| 500/500 [07:29<00:00,  1.11it/s]


Epoch 1/15, Loss: 0.04684797509177588


Validation Epoch 1/15: 100%|██████████| 63/63 [00:45<00:00,  1.39it/s]


Validation Loss: 0.0011589016057624822


Epoch 2/15: 100%|██████████| 500/500 [06:01<00:00,  1.38it/s]


...

Validation Loss: 0.00012755083448682275


Epoch 15/15: 100%|██████████| 500/500 [06:02<00:00,  1.38it/s]


Epoch 15/15, Loss: 0.0001127069442300126


Validation Epoch 15/15: 100%|██████████| 63/63 [00:41<00:00,  1.53it/s]

Validation Loss: 0.00012498767607631013
CPU times: user 3h 7min 57s, sys: 14.4 s, total: 3h 8min 11s
Wall time: 1h 43min 8s


In [14]:
import os

# Define the path to save the model
save_path = "/kaggle/working/vqa_model_new.pth"

# Save the model
torch.save(model.state_dict(), save_path)

# Check if the file is saved
if os.path.exists(save_path):
    print("Model saved successfully at:", save_path)
else:
    print("Failed to save the model.")

Model saved successfully at: /kaggle/working/vqa_model_new.pth


In [16]:
def decode_predictions(outputs, tokenizer):
    probabilities = torch.softmax(outputs, dim=-1)
    predicted_indices = torch.argmax(probabilities, dim=-1)
    predictions = []
    for indices in predicted_indices:
        tokens = tokenizer.convert_ids_to_tokens(indices.tolist())
        prediction = "".join(tokens)
        predictions.append(prediction)
    return predictions

In [18]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
def compute_accuracy(predictions, labels):
    correct = sum(p == l for p, l in zip(predictions, labels))
    total = len(labels)
    accuracy = correct / total
    return accuracy
from sklearn.metrics import precision_recall_fscore_support



def compute_precision_recall_f1(predictions, labels):
    # Ensure both predictions and labels have the same data type
    predictions = np.array(predictions)
    labels = np.array(labels)

    # Convert all elements to integers or strings to ensure consistency
    if predictions.dtype != labels.dtype:
        if np.issubdtype(predictions.dtype, np.number) and np.issubdtype(labels.dtype, np.number):
            predictions = predictions.astype(int)
            labels = labels.astype(int)
        else:
            predictions = predictions.astype(str)
            labels = labels.astype(str)
    
    # Compute precision, recall, and f1 scores
    micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(labels, predictions, average='micro')
    macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    
    return micro_precision, micro_recall, micro_f1, macro_precision, macro_recall, macro_f1

In [20]:
# Set the model to evaluation mode
model.eval()

# Define lists to store predictions and ground truth labels
all_predictions = []
all_labels = []

# Iterate through the test dataset
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        image = batch["image_features"].to(device)
        labels = batch["label"]
#         print(labels)
        
        # Forward pass
        outputs = model(image, input_ids, attention_mask)
#         print(torch.argmax(outputs, dim=-1))
        # Decode predictions
        predictions = decode_predictions(outputs,tokenizer) 
        lbl = []
        for indices in labels:
            tokens = tokenizer.convert_ids_to_tokens(indices.tolist())
            prediction = "".join(tokens)
            lbl.append(prediction)
        # Collect predictions and ground truth labels
        all_predictions.extend(predictions)
        all_labels.extend(lbl)

# Compute evaluation metrics
for i in range(0,len(all_labels)):
    print("Prediction:",all_predictions[i],"Ground Truth:",all_labels[i])
# print(all_predictions)
# print(all_labels)
accuracy = compute_accuracy(np.array(all_predictions), np.array(all_labels))
micro_precision, micro_recall, micro_f1,macro_precision, macro_recall, macro_f1 = compute_precision_recall_f1(np.array(all_predictions), np.array(all_labels))

print(f"Accuracy: {accuracy:.4f}")
print(f"Micro Precision: {micro_precision:.4f}")
print(f"Micro Recall: {micro_recall:.4f}")
print(f"Micro F1 Score: {micro_f1:.4f}")
print(f"Macro Precision: {micro_precision:.4f}")
print(f"Macro Recall: {micro_recall:.4f}")
print(f"Macro F1 Score: {micro_f1:.4f}")

/tmp/ipykernel_35/3601849629.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(item["input_ids"])
/tmp/ipykernel_35/3601849629.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(item["attention_mask"])


Prediction: no Ground Truth: yes
Prediction: [UNK] Ground Truth: table
Prediction: white Ground Truth: white
Prediction: yes Ground Truth: no
Prediction: no Ground Truth: no
Prediction: black Ground Truth: yellow
Prediction: sunny Ground Truth: sunny
Prediction: no Ground Truth: yes
Prediction: wood Ground Truth: wood
Prediction: [UNK] Ground Truth: eagle
Prediction: yes Ground Truth: brick
Prediction: solid Ground Truth: [UNK]
Prediction: grass Ground Truth: umbrella
Prediction: [UNK] Ground Truth: [UNK]
Prediction: no Ground Truth: no
Prediction: grass Ground Truth: catcher
Prediction: [UNK] Ground Truth: knife
Prediction: tennis Ground Truth: [UNK]
Prediction: no Ground Truth: yes
Prediction: no Ground Truth: yes
Prediction: black Ground Truth: black
Prediction: no Ground Truth: 10
Prediction: [UNK] Ground Truth: [UNK]
Prediction: white Ground Truth: white
Prediction: [UNK] Ground Truth: [UNK]
Prediction: yes Ground Truth: no
Prediction: no Ground Truth: no
Prediction: no Ground Tru

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
